In [1]:
import os
import shutil

In [2]:
sourcedir = r"F:\johncooper\20240709_byc\20240709_byc_3"
destdir = r"F:\johncooper\20240709_byc\20240709_byc_2"
# offset is 1 + the last timepoint index in the previous output
# directory, typically ending in _1
offset = 385
n_frame_number_digits = 9
query = 'time'
source_pos_paths = [os.path.join(sourcedir, file) for file in os.listdir(sourcedir) if os.path.isdir(os.path.join(sourcedir, file))]
dest_pos_paths = [os.path.join(destdir, file) for file in os.listdir(destdir) if os.path.isdir(os.path.join(destdir, file))]

In [8]:
# The final line of code that actually moves the files to their renamed
# is commented out so you can print the old and new filepaths next to each
# other and confirm they are correct.
%%capture file_renaming_output

for pos_path in source_pos_paths[1:]:
    filenames = [file for file in os.listdir(pos_path) if file[-4:]=='.tif']
    filepaths = [os.path.join(pos_path, name) for name in filenames]
    
    for old_filepath in filepaths:
        # find the timepoint (frame number) in the original filename 
        query_start = old_filepath.rindex(query)
        frame_number_start = query_start + len(query)
        frame_number_end = frame_number_start + n_frame_number_digits
        frame_number = old_filepath[frame_number_start:frame_number_end]
        # Add an offset to the filename so that it follows in series to the original
        # dataset sitting in destidr
        new_number = str(int(frame_number) + offset).zfill(n_frame_number_digits)
        
        new_filepath = old_filepath.replace(frame_number, new_number)
        new_filepath = new_filepath.replace(sourcedir, destdir)
        # If I delete positions from a second acquisition run, then the filenames will go
        # back to 0 based indexing. So the first position of a new data set, eg. position 13
        # in multiple positions setter, becomes position000 in the filename that gets written
        # to the Pos13 directorys
        xy_n_digits = 3
        xy_dir_query = 'Pos'
        xy_start_from_xydir = pos_path.rindex(xy_dir_query)
        xy_from_dirname = int(pos_path[xy_start_from_xydir + len(xy_dir_query):xy_start_from_xydir + len(xy_dir_query) + xy_n_digits])
        xy_fn_query = 'position'
        xy_start_from_fn = old_filepath.rindex(xy_fn_query)
        xy_from_filename = int(old_filepath[xy_start_from_fn + len(xy_fn_query):xy_start_from_fn + len(xy_fn_query) + xy_n_digits])

        to_replace = xy_fn_query + str(xy_from_filename).zfill(3)
        replace_with = xy_fn_query + str(xy_from_dirname).zfill(3)
        new_filepath = new_filepath.replace(to_replace, replace_with)
        # os.rename(old_filepath, new_filepath)
        print(old_filepath)
        print(new_filepath)
        # shutil.move(old_filepath, new_filepath)